In [33]:
# -- coding:UTF-8 --
import requests
import os
import time
import ssl
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import pymysql
import re


# used to fix Python SSL CERTIFICATE_VERIFY_FAILED
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
####################################

#Dayuan 18 LongTan 21 GuanYin 19 JhongLi 68 PingZhen 20 TaoYuan 17
#So2 1 CO 2 O3 3 PM10 4 NOx 5 NO 6 NO2 7 WIND_SPEED 10 WIND_DIRECT 11 AMB_TEMP 14 RAINFALL 23 RH 38 WS_HR 143 WD_HR 144


In [34]:
url = 'https://taqm.epa.gov.tw/taqm/tw/HourlyData.aspx'
today = datetime.datetime.now()

In [35]:
def main():

    post_component = post_web_page(url)
    LongTan_pollution = LongTan(today,post_component[0],post_component[1],post_component[2])
    LT_PM25 = LongTan_pm25()
    pm25_dict = {'PM2.5':LT_PM25}
    LongTan_Final = LongTan_pollution.copy()
    LongTan_Final.update(pm25_dict)
    print(LongTan_Final)


In [36]:
def post_web_page(url):

    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html5lib')
    view_state = soup.find(id='__VIEWSTATE')['value']
    event_validation = soup.find(id='__EVENTVALIDATION')['value']
    viewstate_generator = soup.find(id='__VIEWSTATEGENERATOR')['value']

    return view_state,event_validation,viewstate_generator

In [37]:
def LongTan_pm25():
    
    URL = 'https://taqm.epa.gov.tw/pm25/tw/HourlyData.aspx'
    resp = requests.get(URL)
    soup = BeautifulSoup(resp.text, 'html5lib')

    view_state = soup.find(id='__VIEWSTATE')['value']
    event_validation = soup.find(id='__EVENTVALIDATION')['value']
    viewstate_generator = soup.find(id='__VIEWSTATEGENERATOR')['value']
    form_data = {
        '__VIEWSTATE': view_state,
        '__VIEWSTATEGENERATOR': viewstate_generator,
        '__EVENTVALIDATION': event_validation,
        'ctl08$lbSite': '21',
        'ctl08$lbParam': '33',
        'ctl08$txtDateS': '{}/{}/{}'.format(today.year,today.month,today.day),
        'ctl08$txtDateE': '{}/{}/{}'.format(today.year,today.month,today.day),
        'ctl08$btnQuery': '查詢即時值'
    }
    resp = requests.post(URL, data=form_data)
    soup = BeautifulSoup(resp.text, 'html5lib')

    for t in soup.find_all('table', 'TABLE_G'):

        input_str = [s for s in t.stripped_strings][26 + today.hour - 1]
        pm25 = input_str

    return pm25

In [38]:
def LongTan(today,view_state,event_validation,viewstate_generator):

    station_num = 21
    polltion_num = [1, 2, 3, 4, 5, 6, 7, 10, 11, 14, 23, 38, 143, 144]
    pollution_name = ['SO2', 'CO', 'O3', 'PM10', 'NOx', 'NO', 'NO2', 'WIND_SPEED', 'WIND_DIRECT', 'AMB_TEMP', \
                      'RAINFALL', 'RH', 'WS_HR', 'WD_HR' ]
    pollution = []

    for p in polltion_num:

        form_data = {
            '__VIEWSTATE': view_state,
            '__VIEWSTATEGENERATOR': viewstate_generator,
            '__EVENTVALIDATION': event_validation,
            'ctl05$lbSite': station_num,
            'ctl05$lbParam': '{}'.format(p),
            'ctl05$txtDateS': '{}/{}/{}'.format(today.year, today.month, today.day),
            'ctl05$txtDateE': '{}/{}/{}'.format(today.year, today.month, today.day),
            'ctl05$btnQuery': '查詢即時值'
        }

        resp = requests.post(url, data=form_data)
        soup = BeautifulSoup(resp.text, 'html5lib')

        for t in soup.find_all('table', 'TABLE_G'):

            fetch_result = [s for s in t.stripped_strings][26 + today.hour - 1]

            pollution.append(fetch_result)

    pollution_result = pollution

    list_dict = zip(pollution_name,pollution_result)
    LongTan_pollution_dict = dict(list_dict)

    return LongTan_pollution_dict

In [39]:
if __name__ == '__main__':
    main()

{'SO2': '3.7', 'CO': '0.46', 'O3': '36', 'PM10': '18', 'NOx': '15', 'NO': '4', 'NO2': '11', 'WIND_SPEED': '1.4', 'WIND_DIRECT': '14', 'AMB_TEMP': '17.6', 'RAINFALL': 'NR', 'RH': '84', 'WS_HR': '1.2', 'WD_HR': '13', 'PM2.5': '16'}
